# Gymnasium CarRacing-v2 with PPO (Vision-based RL)

In this notebook, we train a **vision-based** reinforcement learning agent on the `CarRacing-v2` environment using **Proximal Policy Optimization (PPO)** from Stable-Baselines3.

Goals:

- Use image observations (96×96×3 RGB or preprocessed variants).
- Train PPO with a **shared training schedule** consistent with the team (e.g., 0 → 300k → 600k → 1M steps).
- Log the **reward per episode** and visualize training curves.
- Evaluate **mean reward and variance** at different training stages (checkpoints).
- For each checkpoint, record a **demo video** of the agent driving.


# Import the Packages

In [8]:
# If you haven't installed the dependencies, run this once:

# !pip install "gymnasium[box2d]"
# !pip install stable-baselines3[extra]
# !pip install pygame
# !pip install imageio[ffmpeg]

# Import the Packages

In [38]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

import imageio
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback

import torch

print("Imports successful.")



Imports successful.


# Device Detection (GPU / MPS / CPU)

In [14]:
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using Apple Silicon MPS")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using CUDA GPU:", torch.cuda.get_device_name(0))
else:
    DEVICE = torch.device("cpu")
    print("Using CPU only")

Using Apple Silicon MPS


# 🏎️ Car Racing v2 Environment
- Documentation https://gymnasium.farama.org/environments/box2d/car_racing/#description

In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

import imageio
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback

import torch

print("Imports successful.")




Imports successful.


## Model: PPO


In [34]:
import os

ALGO_NAME = "PPO"

# Go from PPO/ up one level to the project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

LOG_DIR   = os.path.join(PROJECT_ROOT, f"logs_{ALGO_NAME}")
MODEL_DIR = os.path.join(PROJECT_ROOT, f"models_{ALGO_NAME}")
VIDEO_DIR = os.path.join(PROJECT_ROOT, f"videos_{ALGO_NAME}")

# Create the folders only if they don't already exist
for d in [LOG_DIR, MODEL_DIR, VIDEO_DIR]:
    if not os.path.isdir(d):
        os.makedirs(d, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("LOG_DIR:", LOG_DIR)
print("MODEL_DIR:", MODEL_DIR)
print("VIDEO_DIR:", VIDEO_DIR)


PROJECT_ROOT: /Users/xtcmir/Downloads/Gym-CarRacing-RL-main 2
LOG_DIR: /Users/xtcmir/Downloads/Gym-CarRacing-RL-main 2/logs_PPO
MODEL_DIR: /Users/xtcmir/Downloads/Gym-CarRacing-RL-main 2/models_PPO
VIDEO_DIR: /Users/xtcmir/Downloads/Gym-CarRacing-RL-main 2/videos_PPO


In [40]:
carracing_hparams = {
    "total_timesteps": 1_000_000,
    "checkpoint_steps": [100_000, 300_000, 600_000, 1_000_000],

    # PPO hyperparameters
    "n_steps": 2048,
    "batch_size": 64,
    "learning_rate": 3e-4,
    "gamma": 0.99,
    "gae_lambda": 0.95,
    "clip_range": 0.2,
    "ent_coef": 0.01,
    "vf_coef": 0.5,

    # env
    "env_id": "CarRacing-v3",

    # live plot
    "plot_every": 1,
    "moving_avg_window": 5,
}




In [41]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

def make_train_env():
    def _make():
        env = gym.make(carracing_hparams["env_id"], render_mode=None)

        # Just log with Monitor (no resize/grayscale/framestack for now)
        env = Monitor(env, filename=os.path.join(LOG_DIR, "monitor.csv"))
        return env

    return DummyVecEnv([_make])


def make_eval_env(render_mode=None):
    def _make():
        env = gym.make(carracing_hparams["env_id"], render_mode=render_mode)

        env = Monitor(env)  # no filename so we don't overwrite training log
        return env

    return DummyVecEnv([_make])


train_env = make_train_env()
print("Train env created.")
print("Observation space:", train_env.observation_space)
print("Action space:", train_env.action_space)



Train env created.
Observation space: Box(0, 255, (96, 96, 3), uint8)
Action space: Box([-1.  0.  0.], 1.0, (3,), float32)


In [42]:
model = PPO(
    "CnnPolicy",
    train_env,
    n_steps=carracing_hparams["n_steps"],
    batch_size=carracing_hparams["batch_size"],
    learning_rate=carracing_hparams["learning_rate"],
    gamma=carracing_hparams["gamma"],
    gae_lambda=carracing_hparams["gae_lambda"],
    clip_range=carracing_hparams["clip_range"],
    ent_coef=carracing_hparams["ent_coef"],
    vf_coef=carracing_hparams["vf_coef"],
    verbose=1,
    tensorboard_log=LOG_DIR,
    device=DEVICE,
)

print("PPO model created.")


Using mps device
Wrapping the env in a VecTransposeImage.
PPO model created.


# Training: save checkpoints at multiple stages

We train the SAC agent for a total of **1M timesteps**, and save checkpoints at:

- 100k steps  
- 300k steps  
- 600k steps  
- 1,000k steps  

During training, we do **not spam print rewards**.  
Instead, we rely on the `Monitor` log and later **plot the episode rewards**.

In [43]:
class CheckpointCallback(BaseCallback):
    def __init__(self, save_freqs, model_dir, verbose=1):
        super().__init__(verbose)
        self.save_freqs = save_freqs
        self.model_dir = model_dir

    def _on_step(self) -> bool:
        if self.num_timesteps in self.save_freqs:
            save_path = os.path.join(self.model_dir, f"ppo_{self.num_timesteps}_steps.zip")
            self.model.save(save_path)
            if self.verbose:
                print(f"📦 Saved checkpoint: {save_path}")
        return True


In [44]:
checkpoint_callback = CheckpointCallback(
    save_freqs=carracing_hparams["checkpoint_steps"],
    model_dir=MODEL_DIR,
    verbose=1
)


In [ ]:
model.learn(
    total_timesteps=carracing_hparams["total_timesteps"],
    callback=checkpoint_callback,
    progress_bar=True
)


Logging to /Users/xtcmir/Downloads/Gym-CarRacing-RL-main 2/logs_PPO/PPO_2
